In [1]:
from keras.models import Sequential
from keras.layers import SimpleRNN

model=Sequential()
model.add(SimpleRNN(3,input_shape=(2,10)))
#model.add(SimpleRNN(3,input_length=2,input_dim=10))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [1]:
#LSTM
#RNN : 이전의 것들을 바탕으로 이해하고 예측하자.
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN,LSTM,Bidirectional

train_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
print(np.shape(train_X))

(4, 5)


In [5]:
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
rnn = SimpleRNN(3)
# rnn = SimpleRNN(3, return_sequences=False, return_state=False)와 동일.
hidden_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[ 0.7180495  -0.96071225  0.40731433]], shape: (1, 3)


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""
t=Tokenizer()
t.fit_on_texts([text])
vocab_size=len(t.word_index)+1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [12]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [14]:
sequences=list()
for line in text.split('\n'):
    encoded=t.texts_to_sequences([line])[0]
    for i in range(1,len(encoded)):
        sequence=encoded[:i+1]
        sequences.append(sequence)
print("학습에 사용할 샘플의 개수: %d"%len(sequences))

학습에 사용할 샘플의 개수: 11


In [15]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [16]:
max_len=max(len(l) for l in sequences)#가장 긴 샘플의 길이
sequences=pad_sequences(sequences,maxlen=max_len,padding='pre')#길이를 6으로 패딩한다.
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [17]:
sequences=np.array(sequences)
#샘플의 마지막 단어를 레이블로 분리하자.
X=sequences[:,:-1]
y=sequences[:,-1]
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [18]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [19]:
y=to_categorical(y,num_classes=vocab_size)
print(y) #원핫인코딩으로

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [20]:
#모델 설계하기
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,SimpleRNN
model=Sequential()
model.add(Embedding(vocab_size,10,input_length=max_len-1))#레이블을 분리했으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=200,verbose=2)

Epoch 1/200
1/1 - 2s - loss: 2.4855 - accuracy: 0.2727
Epoch 2/200
1/1 - 0s - loss: 2.4718 - accuracy: 0.2727
Epoch 3/200
1/1 - 0s - loss: 2.4580 - accuracy: 0.3636
Epoch 4/200
1/1 - 0s - loss: 2.4439 - accuracy: 0.4545
Epoch 5/200
1/1 - 0s - loss: 2.4296 - accuracy: 0.4545
Epoch 6/200
1/1 - 0s - loss: 2.4149 - accuracy: 0.3636
Epoch 7/200
1/1 - 0s - loss: 2.3998 - accuracy: 0.3636
Epoch 8/200
1/1 - 0s - loss: 2.3841 - accuracy: 0.3636
Epoch 9/200
1/1 - 0s - loss: 2.3678 - accuracy: 0.3636
Epoch 10/200
1/1 - 0s - loss: 2.3509 - accuracy: 0.4545
Epoch 11/200
1/1 - 0s - loss: 2.3334 - accuracy: 0.4545
Epoch 12/200
1/1 - 0s - loss: 2.3150 - accuracy: 0.4545
Epoch 13/200
1/1 - 0s - loss: 2.2960 - accuracy: 0.4545
Epoch 14/200
1/1 - 0s - loss: 2.2761 - accuracy: 0.4545
Epoch 15/200
1/1 - 0s - loss: 2.2555 - accuracy: 0.4545
Epoch 16/200
1/1 - 0s - loss: 2.2341 - accuracy: 0.4545
Epoch 17/200
1/1 - 0s - loss: 2.2119 - accuracy: 0.4545
Epoch 18/200
1/1 - 0s - loss: 2.1890 - accuracy: 0.4545
E

Epoch 147/200
1/1 - 0s - loss: 0.3792 - accuracy: 0.9091
Epoch 148/200
1/1 - 0s - loss: 0.3746 - accuracy: 0.9091
Epoch 149/200
1/1 - 0s - loss: 0.3700 - accuracy: 0.9091
Epoch 150/200
1/1 - 0s - loss: 0.3655 - accuracy: 0.9091
Epoch 151/200
1/1 - 0s - loss: 0.3611 - accuracy: 0.9091
Epoch 152/200
1/1 - 0s - loss: 0.3567 - accuracy: 0.9091
Epoch 153/200
1/1 - 0s - loss: 0.3525 - accuracy: 0.9091
Epoch 154/200
1/1 - 0s - loss: 0.3483 - accuracy: 0.9091
Epoch 155/200
1/1 - 0s - loss: 0.3442 - accuracy: 1.0000
Epoch 156/200
1/1 - 0s - loss: 0.3401 - accuracy: 1.0000
Epoch 157/200
1/1 - 0s - loss: 0.3362 - accuracy: 1.0000
Epoch 158/200
1/1 - 0s - loss: 0.3322 - accuracy: 1.0000
Epoch 159/200
1/1 - 0s - loss: 0.3284 - accuracy: 1.0000
Epoch 160/200
1/1 - 0s - loss: 0.3246 - accuracy: 1.0000
Epoch 161/200
1/1 - 0s - loss: 0.3209 - accuracy: 1.0000
Epoch 162/200
1/1 - 0s - loss: 0.3172 - accuracy: 1.0000
Epoch 163/200
1/1 - 0s - loss: 0.3136 - accuracy: 1.0000
Epoch 164/200
1/1 - 0s - loss: 

In [21]:
def sentence_generation(model,t,current_word,n):#모델,토크나이저, 현재단어, 반복할 횟수
    init_word=current_word
    sentence=''
    for _ in range(n):#n번번복
        encoded=t.texts_to_sequences([current_word])[0] #현재 단어에 대한 정수 인코딩
        encoded=pad_sequences([encoded],maxlen=5,padding='pre')#데이터에 대한 패딩
        result=model.predict_classes(encoded,verbose=0)
        for word, index in t.word_index.items():
            if index==result:
                break
        current_word=current_word+' '+word#현재단어+ " " + 예측 단어를 현대단어로 바꿈
        sentence=sentence+' '+word#예측단어를 문장에 저장
    sentence=init_word+sentence
    return sentence
print(sentence_generation(model,t,'경마장에',4))

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


경마장에 있는 말이 뛰고 있다


In [22]:
print(sentence_generation(model,t,'그의',2))

그의 말이 법이다


In [24]:
print(sentence_generation(model,t,'가는',5))

가는 말이 고와야 오는 말이 곱다


In [25]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

df=pd.read_csv(r'C:\Users\Msi\Desktop\ArticlesApril2018.csv')

In [26]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [27]:
df['headline'].isnull().values.any()

False

In [28]:
headline=[]#리스트 선언
headline.extend(list(df.headline.values))#헤드라인값들을 리스트로 저장
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [29]:
print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 1324


In [30]:
headline=[n for n in headline if n !="Unknown"] #"Unknown제거"
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

노이즈값 제거 후 샘플의 개수 : 1214


In [31]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [33]:
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()#구두점 제거와 동시에 소문자화
text=[repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [34]:
t=Tokenizer()
t.fit_on_texts(text)
vocab_size=len(t.word_index)+1
print("단어집합의 크기:%d"%vocab_size)

단어집합의 크기:3494


In [36]:
sequences=list()
for line in text:
    encoded=t.texts_to_sequences([line])[0]
    for i in range(1,len(encoded)):
        sequence=encoded[:i+1]
        sequences.append(sequence)
        
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [37]:
index_to_word={}
for key, value in t.word_index.items():
    index_to_word[value]=key
print('빈도수 상위 582번 단어: {}'.format(index_to_word[582]))

빈도수 상위 582번 단어: offer


In [39]:
max_len=max(len(l) for l in sequences)
sequences=pad_sequences(sequences,maxlen=max_len,padding='pre')
print(sequences[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   99  269  371 1115  582]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   99  269  371 1115  582   52]]


In [40]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [41]:
print(y[:3]) #레이블

[ 269  371 1115]


In [42]:
y=to_categorical(y,num_classes=vocab_size) #원-핫인코딩

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM

model=Sequential()
model.add(Embedding(vocab_size,10,input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=200,verbose=2)

Epoch 1/200
244/244 - 4s - loss: 7.6449 - accuracy: 0.0274
Epoch 2/200
244/244 - 2s - loss: 7.1108 - accuracy: 0.0319
Epoch 3/200
244/244 - 2s - loss: 6.9734 - accuracy: 0.0386
Epoch 4/200
244/244 - 2s - loss: 6.8527 - accuracy: 0.0413
Epoch 5/200
244/244 - 2s - loss: 6.7159 - accuracy: 0.0438
Epoch 6/200
244/244 - 2s - loss: 6.5581 - accuracy: 0.0487
Epoch 7/200
244/244 - 2s - loss: 6.3726 - accuracy: 0.0518
Epoch 8/200
244/244 - 2s - loss: 6.1760 - accuracy: 0.0549
Epoch 9/200
244/244 - 2s - loss: 5.9793 - accuracy: 0.0600
Epoch 10/200
244/244 - 2s - loss: 5.7914 - accuracy: 0.0669
Epoch 11/200
244/244 - 2s - loss: 5.6083 - accuracy: 0.0706
Epoch 12/200
244/244 - 2s - loss: 5.4380 - accuracy: 0.0756
Epoch 13/200
244/244 - 2s - loss: 5.2752 - accuracy: 0.0800
Epoch 14/200
244/244 - 2s - loss: 5.1185 - accuracy: 0.0897
Epoch 15/200
244/244 - 2s - loss: 4.9703 - accuracy: 0.1005
Epoch 16/200
244/244 - 2s - loss: 4.8301 - accuracy: 0.1096
Epoch 17/200
244/244 - 2s - loss: 4.6936 - accura

244/244 - 3s - loss: 0.3454 - accuracy: 0.9145
Epoch 138/200
244/244 - 2s - loss: 0.3444 - accuracy: 0.9135
Epoch 139/200
244/244 - 2s - loss: 0.3641 - accuracy: 0.9100
Epoch 140/200
244/244 - 2s - loss: 0.3359 - accuracy: 0.9149
Epoch 141/200
244/244 - 2s - loss: 0.3249 - accuracy: 0.9170
Epoch 142/200
244/244 - 2s - loss: 0.3211 - accuracy: 0.9159
Epoch 143/200
244/244 - 2s - loss: 0.3187 - accuracy: 0.9164
Epoch 144/200
244/244 - 2s - loss: 0.3166 - accuracy: 0.9164
Epoch 145/200
244/244 - 2s - loss: 0.3144 - accuracy: 0.9161
Epoch 146/200
244/244 - 2s - loss: 0.3117 - accuracy: 0.9157
Epoch 147/200
244/244 - 2s - loss: 0.3091 - accuracy: 0.9157
Epoch 148/200
244/244 - 2s - loss: 0.3078 - accuracy: 0.9162
Epoch 149/200
244/244 - 2s - loss: 0.3055 - accuracy: 0.9141
Epoch 150/200
244/244 - 2s - loss: 0.3053 - accuracy: 0.9150
Epoch 151/200
244/244 - 2s - loss: 0.3030 - accuracy: 0.9163
Epoch 152/200
244/244 - 2s - loss: 0.3046 - accuracy: 0.9166
Epoch 153/200
244/244 - 2s - loss: 0.3

In [45]:
def sentence_generation(model,t,current_word,n):#모델, 토크나이저, 현재단어, 반복할 횟수
    init_word=current_word#처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence=''
    for _ in range(n):
        encoded=t.texts_to_sequences([current_word])[0]
        encoded=pad_sequences([encoded],maxlen=23,padding='pre')
        result=model.predict_classes(encoded,verbose=0)
        for word,index in t.word_index.items():
            if index == result:
                break
        current_word=current_word + ' '+word
        sentence=sentence+ ' '+word
    sentence=init_word+sentence
    return sentence
print(sentence_generation(model,t,'i',10))

i disapprove of school vouchers can i still apply for them


In [46]:
print(sentence_generation(model, t, 'how', 10))

how to make a crossword puzzle pay just from the big
